In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.datasets import load_files 
from sklearn.model_selection import train_test_split
from glob import glob


from keras.utils import np_utils

In [27]:
def load_dataset(path):
    data = load_files(container_path=path, shuffle=True)
    flower_files = np.array(data['filenames'])
    flower_targets = np_utils.to_categorical(np.array(data['target']), 5)
    return flower_files, flower_targets

flower_files, flower_targets = load_dataset('./flowers')
train_files = flower_files[:2700]
valid_files = flower_files[2700:3500]
test_files = flower_files[3500:]

train_targets = flower_targets[:2700]
valid_targets = flower_targets[2700:3500]
test_targets = flower_targets[3500:]

flower_names = [item for item in sorted(glob("./flowers/*/"))]
print('There are %d total flower categories.' % len(flower_names))
print('There are %s total flower images.\n' % len(flower_files))
print('There are %d training flower images.' % len(train_files))
print('There are %d validation flower images.' % len(valid_files))
print('There are %d test flower images.' % len(test_files))

There are 5 total flower categories.
There are 4323 total flower images.

There are 2700 training flower images.
There are 800 validation flower images.
There are 823 test flower images.


In [28]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [29]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [30]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

In [31]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255


 98%|███████████████████████████████████████████████████████████████████████████▎ | 2641/2700 [00:14<00:00, 177.85it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 823/823 [00:05<00:00, 145.73it/s]

In [32]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
print(train_tensors.shape)
### TODO: Define your architecture.
model.add(Conv2D(filters=16, kernel_size=2, strides=1, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=32, kernel_size=2, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=64, kernel_size=2, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))
model.summary()

(2700, 224, 224, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 27, 27, 64)        0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 64)                0

In [33]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [34]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 5

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 2700 samples, validate on 800 samples
Epoch 1/5
2700/2700 [==============================] - ETA: 5:00 - loss: 1.5985 - acc: 0.400 - ETA: 3:16 - loss: 1.5939 - acc: 0.325 - ETA: 2:41 - loss: 1.5896 - acc: 0.283 - ETA: 2:20 - loss: 1.6096 - acc: 0.250 - ETA: 2:07 - loss: 1.6086 - acc: 0.260 - ETA: 1:57 - loss: 1.6057 - acc: 0.258 - ETA: 1:50 - loss: 1.6073 - acc: 0.235 - ETA: 1:46 - loss: 1.6030 - acc: 0.243 - ETA: 1:43 - loss: 1.5969 - acc: 0.255 - ETA: 1:41 - loss: 1.5948 - acc: 0.255 - ETA: 1:38 - loss: 1.6002 - acc: 0.254 - ETA: 1:35 - loss: 1.6023 - acc: 0.250 - ETA: 1:34 - loss: 1.6009 - acc: 0.242 - ETA: 1:32 - loss: 1.6013 - acc: 0.242 - ETA: 1:31 - loss: 1.6022 - acc: 0.240 - ETA: 1:29 - loss: 1.6012 - acc: 0.243 - ETA: 1:29 - loss: 1.5998 - acc: 0.241 - ETA: 1:28 - loss: 1.6003 - acc: 0.238 - ETA: 1:27 - loss: 1.5989 - acc: 0.236 - ETA: 1:25 - loss: 1.5988 - acc: 0.232 - ETA: 1:25 - loss: 1.5962 - acc: 0.240 - ETA: 1:23 - loss: 1.5938 - acc: 0.240 - ETA: 1:23 - loss: 

2700/2700 [==============================] - ETA: 1:36 - loss: 1.1853 - acc: 0.450 - ETA: 1:35 - loss: 1.2410 - acc: 0.425 - ETA: 1:37 - loss: 1.2579 - acc: 0.400 - ETA: 1:37 - loss: 1.2034 - acc: 0.425 - ETA: 1:35 - loss: 1.2130 - acc: 0.440 - ETA: 1:34 - loss: 1.2021 - acc: 0.458 - ETA: 1:34 - loss: 1.1824 - acc: 0.471 - ETA: 1:34 - loss: 1.2261 - acc: 0.437 - ETA: 1:32 - loss: 1.2437 - acc: 0.433 - ETA: 1:31 - loss: 1.2325 - acc: 0.445 - ETA: 1:29 - loss: 1.2196 - acc: 0.454 - ETA: 1:28 - loss: 1.2227 - acc: 0.458 - ETA: 1:27 - loss: 1.2169 - acc: 0.453 - ETA: 1:26 - loss: 1.2408 - acc: 0.421 - ETA: 1:25 - loss: 1.2513 - acc: 0.420 - ETA: 1:24 - loss: 1.2599 - acc: 0.415 - ETA: 1:23 - loss: 1.2677 - acc: 0.420 - ETA: 1:22 - loss: 1.2700 - acc: 0.419 - ETA: 1:21 - loss: 1.2698 - acc: 0.426 - ETA: 1:21 - loss: 1.2724 - acc: 0.430 - ETA: 1:21 - loss: 1.2701 - acc: 0.433 - ETA: 1:20 - loss: 1.2706 - acc: 0.434 - ETA: 1:19 - loss: 1.2657 - acc: 0.437 - ETA: 1:19 - loss: 1.2623 - acc: 0.4

2700/2700 [==============================] - ETA: 1:49 - loss: 1.2218 - acc: 0.400 - ETA: 1:44 - loss: 1.1843 - acc: 0.450 - ETA: 1:43 - loss: 1.1597 - acc: 0.483 - ETA: 1:41 - loss: 1.1197 - acc: 0.525 - ETA: 1:40 - loss: 1.1055 - acc: 0.500 - ETA: 1:40 - loss: 1.1194 - acc: 0.466 - ETA: 1:41 - loss: 1.1410 - acc: 0.500 - ETA: 1:41 - loss: 1.1670 - acc: 0.500 - ETA: 1:41 - loss: 1.1858 - acc: 0.483 - ETA: 1:41 - loss: 1.1781 - acc: 0.495 - ETA: 1:41 - loss: 1.2060 - acc: 0.472 - ETA: 1:43 - loss: 1.2028 - acc: 0.475 - ETA: 1:43 - loss: 1.1785 - acc: 0.492 - ETA: 1:44 - loss: 1.1801 - acc: 0.485 - ETA: 1:43 - loss: 1.1939 - acc: 0.476 - ETA: 1:43 - loss: 1.1980 - acc: 0.490 - ETA: 1:43 - loss: 1.2003 - acc: 0.482 - ETA: 1:42 - loss: 1.1980 - acc: 0.475 - ETA: 1:41 - loss: 1.1996 - acc: 0.481 - ETA: 1:39 - loss: 1.1973 - acc: 0.485 - ETA: 1:38 - loss: 1.1928 - acc: 0.492 - ETA: 1:38 - loss: 1.1956 - acc: 0.495 - ETA: 1:37 - loss: 1.1943 - acc: 0.500 - ETA: 1:36 - loss: 1.1834 - acc: 0.5

In [35]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')


In [36]:
# get index of predicted dog breed for each image in test set
flower_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(flower_breed_predictions)==np.argmax(test_targets, axis=1))/len(flower_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 47.3876%
